In [3]:
import numpy as np  
from sklearn.metrics.pairwise import cosine_similarity  
import pandas as pd



In [4]:
customers = pd.read_csv('Customers.csv')  
transactions = pd.read_csv('Transactions.csv')




In [5]:
# Preparing the lookalike model  
# First, create a Customer-Product matrix  
customer_product = transactions.pivot_table(index='CustomerID', columns='ProductID', values='Quantity', fill_value=0)  

# Calculate similarity matrix  
similarity = cosine_similarity(customer_product)  
similarity_df = pd.DataFrame(similarity, index=customer_product.index, columns=customer_product.index)  

# Function to get top N lookalikes  
def get_lookalikes(customer_id, n=3):  
    similar_indices = similarity_df[customer_id].nlargest(n+1).index[1:]  # Exclude self-reference  
    return similar_indices.tolist()  



In [6]:
# Create Lookalike.csv for first 20 customers  
lookalike_data = {  
    'CustomerID': [],  
    'Lookalike1': [],  
    'Lookalike2': [],  
    'Lookalike3': []  
}


for customer in customers['CustomerID'].head(20):  # First 20 customers  
    lookalikes = get_lookalikes(customer)  
    lookalike_data['CustomerID'].append(customer)  
    lookalike_data['Lookalike1'].append(lookalikes[0] if len(lookalikes) > 0 else None)  
    lookalike_data['Lookalike2'].append(lookalikes[1] if len(lookalikes) > 1 else None)  
    lookalike_data['Lookalike3'].append(lookalikes[2] if len(lookalikes) > 2 else None)  

lookalike_df = pd.DataFrame(lookalike_data)  
lookalike_df.to_csv('Lookalike.csv', index=False)